In [2]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import re
import random
import plotly.graph_objs as go
import chart_studio.plotly as py
import cufflinks
pd.options.display.max_columns = 30
from IPython.core.interactiveshell import InteractiveShell
import plotly.figure_factory as ff
InteractiveShell.ast_node_interactivity = 'all'
from plotly.offline import iplot
cufflinks.go_offline()
cufflinks.set_config_file(world_readable=True, theme='solar')



workbook = xlrd.open_workbook('hotel-list.xls', ignore_workbook_corruption=True)
df = pd.read_excel(workbook)

In [119]:
df = pd.read_csv('CRAWL-2.csv')

In [82]:
def print_description(index):
    example = df[df.index == index][['descriptions', 'names']].values[0]
    if len(example) > 0:
        print(example[0])
        print('Name:', example[1])

In [83]:
print_description(10)

Bạn đủ điều kiện nhận giảm giá Genius tại GREENECO DA LAT HOTEL - Khách sạn Green Eco Đà Lạt! Để tiết kiệm tại chỗ nghỉ này, bạn chỉ cần đăng nhập.
Tọa lạc tại thành phố Đà Lạt, cách Thiền viện Trúc Lâm 2,2 km và Hồ Tuyền Lâm 2,3 km, GreenEco Da Lat Hotel cung cấp chỗ nghỉ với sảnh khách chung và WiFi miễn phí trong toàn bộ khuôn viên cũng như chỗ đỗ xe riêng miễn phí cho khách lái xe. Chỗ nghỉ này nằm trong bán kính 3,5 km từ Quảng trường Lâm Viên, 3,6 km từ Công viên Yersin và 3,6 km từ Hồ Xuân Hương. Nơi đây có dịch vụ phòng, lễ tân 24 giờ và tổ chức các tour du lịch cho khách.
Tất cả phòng nghỉ tại GreenEco Da Lat Hotel được trang bị tủ lạnh, minibar, ấm đun nước, vòi sen, máy sấy tóc, bàn làm việc, ga trải giường và khăn tắm. Mỗi phòng đều được bố trí TV màn hình phẳng, máy điều hòa và phòng tắm riêng với vòi xịt/chậu rửa vệ sinh cùng đồ vệ sinh cá nhân miễn phí. Một số phòng có sân hiên.
Chỗ nghỉ phục vụ bữa sáng à la carte hằng ngày.
Name: GREENECO DA LAT HOTEL - Khách sạn Gree

In [84]:
def get_top_n_words(corpus, n=None):
    vec = CountVectorizer().fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
common_words = get_top_n_words(df['descriptions'], 20)
df1 = pd.DataFrame(common_words, columns = ['descriptions' , 'count'])
df1.groupby('descriptions').sum()['count'].sort_values().iplot(kind='barh', yTitle='Count', linecolor='black', title='Top 20 words in hotel description before removing stop words')

In [85]:
def get_top_n_bigram(corpus, n=None):
    vec = CountVectorizer(ngram_range=(2, 2)).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
common_words = get_top_n_bigram(df['descriptions'], 20)
df3 = pd.DataFrame(common_words, columns = ['descriptions' , 'count'])
df3.groupby('descriptions').sum()['count'].sort_values(ascending=False).iplot(kind='bar', yTitle='Count', linecolor='black', title='Top 20 bigrams in hotel description before removing stop words')

In [86]:
def get_top_n_trigram(corpus, n=None):
    vec = CountVectorizer(ngram_range=(3, 3)).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
common_words = get_top_n_trigram(df['descriptions'], 20)
df5 = pd.DataFrame(common_words, columns = ['descriptions' , 'count'])
df5.groupby('descriptions').sum()['count'].sort_values(ascending=False).iplot(kind='bar', yTitle='Count', linecolor='black', title='Top 20 trigrams in hotel description before removing stop words')

In [87]:
def get_top_n_trigram(corpus, n=None):
    vec = CountVectorizer(ngram_range=(4, 4)).fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
common_words = get_top_n_trigram(df['descriptions'], 20)
df6 = pd.DataFrame(common_words, columns = ['descriptions' , 'count'])
df6.groupby('descriptions').sum()['count'].sort_values(ascending=False).iplot(kind='bar', yTitle='Count', linecolor='black', title='Top 20 trigrams in hotel description before removing stop words')

In [120]:
df['descriptions']=df['descriptions'].apply(lambda x: re.sub('.+đăng nhập.','',x))
df['descriptions_clean']=df['descriptions'].apply(lambda x: re.sub('.+đăng nhập.','',x))

In [121]:
df['descriptions']=df['descriptions'].str.replace('\n','')
df['descriptions_clean']=df['descriptions'].str.replace('\n','')
df['descriptions_clean']=df['descriptions'].str.replace('Tọa lạc tại','')

In [122]:
df['lat']=df['latitude_longitude'].apply(lambda x: x.split(',', 1)[0])
df['lon']=df['latitude_longitude'].apply(lambda x: x.split(',', 1)[1])

In [123]:
df.sample(5)

,links,names,reviews,scores,addresses,descriptions,latitude_longitude,DEST,descriptions_clean,lat,lon
60,https://www.booking.com/hotel/vn/hanoi-prime-c...,Hanoi Prime Center Hotel,NaN,"8,9","36 Hang Huong, Quận Hoàn Kiếm, Hà Nội, Việt...",Tọa lạc tại một vị trí thuận tiện ở thành phố ...,"21.03531900,105.84550500",-3714993,"một vị trí thuận tiện ở thành phố Hà Nội, Han...",21.03531900,105.84550500
130,https://www.booking.com/hotel/vn/cozrum-homes-...,Cozrum Homes - Sonata Residence,NaN,"7,6","41 Nguyễn Thị Thập, Phường Tân Phú, Quận 7, TP...","Set in Ho Chi Minh City, 1.7 km from Saigon Ex...","10.73694600,106.72871900",-3730078,"Set in Ho Chi Minh City, 1.7 km from Saigon Ex...",10.73694600,106.72871900
272,https://www.booking.com/hotel/vn/la-vague.vi.h...,La Vague Hotel,NaN,"8,6","08 Trần Phú 08 Trần Phú, Nha Trang, Việt Nam","Quay mặt ra biển ở thành phố Nha Trang, cách B...","12.22021812,109.20365352",-3723998,"Quay mặt ra biển ở thành phố Nha Trang, cách B...",12.22021812,109.20365352
422,https://www.booking.com/hotel/vn/homestay-ha-l...,Homestay Hạ long,NaN,NaN,"Đường Hoàng Quốc Việt, Hạ Long, Việt Nam","Set in Ha Long, 1.4 km from Marina Bay Beach a...","20.95917022,107.00860572",-3715715,"Set in Ha Long, 1.4 km from Marina Bay Beach a...",20.95917022,107.00860572
129,https://www.booking.com/hotel/vn/la-bonita.vi....,La Bonita Hotel and Apartments,NaN,"8,5","42 Hưng Phước 4, Tân Phong, District 7, Ho Chi...","Tọa lạc tại Thành phố Hồ Chí Minh, cách Trung ...","10.72969700,106.70713000",-3730078,"Thành phố Hồ Chí Minh, cách Trung tâm Hội chợ...",10.72969700,106.70713000


# XỬ LÝ STOP WORD

In [124]:
def get_stopwords_list(stop_file_path):
    """load stop words """
    with open(stop_file_path, 'r', encoding="utf-8") as f:
        stopwords = f.readlines()
        stop_set = set(m.strip() for m in stopwords)
        return list(frozenset(stop_set))

In [125]:
PATH = 'C:/Users/TechCare/Documents/'

In [126]:
stopwords_path = PATH+"vietnamese-stopwords.txt"
stopwords = get_stopwords_list(stopwords_path)

In [127]:
# viết hàm tiền xử lý dữ liệu trên cột overview
def proccessing_data(data):
  #- chuyển về từ thường
    data = data.lower()
  #- Xóa dấu câu, ký tự đặc biệt
    data = re.sub('\W+',' ', data)
  #- xóa stopword
    data =  ' '.join([word for word in data.split() if word not in stopwords])
  #- tách từ
    data = word_tokenize(data,format = 'text')
  #- nối câu
  #data =  ' '.join([word for word in data])
    return data

In [128]:
from tqdm import tqdm

In [129]:
for i in tqdm(range(0,len(df))):
  try:
    df['descriptions_clean'][i] = proccessing_data(df['descriptions_clean'][i])
  except Exception as ex:
    print(df['descriptions'][i])

100%|████████████████████████████████████████████████████████████████████████████████| 450/450 [00:05<00:00, 80.79it/s]


In [130]:
df.sample(5)

,links,names,reviews,scores,addresses,descriptions,latitude_longitude,DEST,descriptions_clean,lat,lon
281,https://www.booking.com/hotel/vn/vinpearl-empi...,Meliá Vinpearl Nha Trang Empire,NaN,"9,1","44 Le Thanh Ton, Nha Trang, Việt Nam",Meliá Vinpearl Nha Trang Empire nằm ở thành ph...,"12.24342120,109.19349690",-3723998,meliá vinpearl nha trang empire nằm thành_phố ...,12.24342120,109.19349690
310,https://www.booking.com/hotel/vn/pilgrimage-vi...,Pilgrimage Village Boutique Resort & Spa,NaN,"9,2","130 Minh Mang Road, Huế, Việt Nam",Pilgrimage Village Boutique Resort & Spa là re...,"16.43540086,107.58060336",-3715887,pilgrimage village boutique resort spa resort ...,16.43540086,107.58060336
446,https://www.booking.com/hotel/vn/the-secret-ha...,The Secret HaLong,NaN,"8,4","E6.64 Sun Premier Village,Bai Chay, Hạ Long, ...","Tọa lạc tại thành phố Hạ Long, cách Bãi biển B...","20.94697400,107.03257200",-3715715,thành_phố hạ long bãi biển bãi cháy 500 m the ...,20.94697400,107.03257200
404,https://www.booking.com/hotel/vn/victory-star-...,Victory Star Cruise,NaN,"8,4","Hon Gai, Hạ Long, Việt Nam","Đi dọc theo Vịnh Hạ Long, Victory Star Cruise ...","20.95298725,107.05607995",-3715715,đi dọc vịnh hạ long victory star cruise cung p...,20.95298725,107.05607995
444,https://www.booking.com/hotel/vn/hallo-thanh-p...,Hallo Hotel,NaN,"8,4","Đại lộ Hạ Long Marine, Hạ Long, Việt Nam","Tọa lạc tại thành phố Hạ Long, nằm trong bán k...","20.95589600,107.01690000",-3715715,thành_phố hạ long nằm kính 3 4 km cáp_treo nữ_...,20.95589600,107.01690000


# TẠO MODEL

In [47]:
def recommendations(names, DEST):

    recommended_hotels = []
    links=[]
    addresses =[]
    scores =[]
    descriptions=[]
    lat=[]
    lon=[]

    
    ks = df[df.names == names]
    
    hotel = df[df.DEST == DEST]
    hotel = pd.concat([ks, hotel], ignore_index = True)
    
    
    #tính tf-idf + cosine_similarities
    tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 4), min_df=0)
    tfidf_matrix = tf.fit_transform(hotel['descriptions_clean'])
    cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)
    #chuyển names thành index
    hotel.set_index('names', inplace = True)
    indices = pd.Series(hotel.index)
    # gettin the index of the hotel that matches the name
    idx = indices[indices == names].index[0]
    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_similarities[idx]).sort_values(ascending = False)

    # getting the indexes of the 10 most similar hotels except itself
    top_10_indexes = list(score_series.iloc[0:11].index)
    
    # populating the list with the names of the top 10 matching hotels
    for i in top_10_indexes:
        recommended_hotels.append(list(hotel.index)[i])
        links.append(list(hotel['links'])[i])
        addresses.append(list(hotel['addresses'])[i])
        scores.append(list(hotel['scores'])[i])
        descriptions.append(list(hotel['descriptions'])[i])
        lat.append(list(hotel['lat'])[i])
        lon.append(list(hotel['lon'])[i])
  
    return pd.DataFrame({'links':links,'names':recommended_hotels,'addresses': addresses,'scores':scores,'Description':descriptions,'Lat':lat,'Lon':lon})
    #return score_series

In [48]:
recommendations('Menora Grand Da Nang',-3712045)

0     1.000000
37    0.097992
6     0.095445
13    0.090976
20    0.087530
38    0.084490
19    0.081015
10    0.073597
46    0.068458
14    0.068050
49    0.067448
27    0.066399
7     0.066203
18    0.065308
26    0.064058
16    0.062169
11    0.060548
23    0.059425
17    0.058853
47    0.058380
43    0.057504
4     0.055902
8     0.055637
1     0.054314
36    0.053439
15    0.050554
12    0.050438
39    0.046939
29    0.046685
42    0.046509
50    0.045807
9     0.043845
2     0.042897
41    0.041528
21    0.037997
25    0.033459
5     0.033326
40    0.033176
44    0.031950
33    0.030489
34    0.030066
35    0.028544
3     0.027013
24    0.026863
48    0.026257
45    0.025289
22    0.023453
31    0.021756
32    0.016718
30    0.011055
28    0.010020
dtype: float64

In [134]:
import pickle

pickle.dump(df,open('df.pkl','wb'))


# RECOMMEND BASE ON  INPUT DESCRIPTION

In [131]:
def recommendations(des, DEST):

    recommended_hotels = []
    links=[]
    addresses =[]
    scores =[]
    descriptions=[]
    lat = []
    lon = []

    
    ks = pd.DataFrame({'links':['1'],'names':['ABC'],'reviews':['1'],'scores':['1'],
                      'addresses': ['1'],'Description':['1'],'latitude_longitude': ['1'],'DEST': ['1'],'descriptions_clean':[des],'lat':['1'],'lon':['1']})
    
    
    hotel = df[df.DEST == DEST]
    hotel = pd.concat([ks, hotel], ignore_index = True)
    
    
    #tính tf-idf + cosine_similarities
    tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 4), min_df=0)
    tfidf_matrix = tf.fit_transform(hotel['descriptions_clean'])
    cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)
    #chuyển names thành index
    hotel.set_index('names', inplace = True)
    indices = pd.Series(hotel.index)
    # gettin the index of the hotel that matches the name
    idx = indices[indices == 'ABC'].index[0]
    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_similarities[idx]).sort_values(ascending = False)

    # getting the indexes of the 10 most similar hotels except itself
    top_10_indexes = list(score_series[score_series>0].index[1:11])
    
    # populating the list with the names of the top 10 matching hotels
    for i in top_10_indexes:
        recommended_hotels.append(list(hotel.index)[i])
        links.append(list(hotel['links'])[i])
        addresses.append(list(hotel['addresses'])[i])
        scores.append(list(hotel['scores'])[i])
        descriptions.append(list(hotel['descriptions'])[i])
        lat.append(list(hotel['lat'])[i])
        lon.append(list(hotel['lon'])[i])
  
    return pd.DataFrame({'links':links,'names':recommended_hotels,'addresses': addresses,'scores':scores,'Description':descriptions,'lat': lat,'lon': lon})

In [133]:
recommendations('biển',-3712045)


,links,names,addresses,scores,Description,lat,lon


In [135]:
df.to_csv('data.csv',header = True, index = False)

# TEST

In [67]:
a[a==0].index[1:11]

Int64Index([38, 28, 29, 30, 31, 32, 33, 34, 35, 36], dtype='int64')

In [26]:
ks = df[df.names == 'Menora Grand Da Nang']
    
hotel = df[df.DEST == -3712045]
hotelx = pd.concat([ks, hotel], ignore_index = True)

In [27]:
hotel

,links,names,reviews,scores,addresses,descriptions,latitude_longitude,DEST,descriptions_clean,lat,lon
0,https://www.booking.com/hotel/vn/bazan-hotel-d...,Bazan Hotel Dalat,NaN,"9,1","36 To Hien Thanh , Đà Lạt, Việt Nam","Tọa lạc tại thành phố Đà Lạt, cách Công viên Y...","11.92824835,108.44512304",-3712045,thành phố đà lạt công viên yersin 1 6 km bazan...,11.92824835,108.44512304
1,https://www.booking.com/hotel/vn/dalat-wind.vi...,Dalat Wind Hotel,NaN,"8,4","Lot R2 03-04. Golf Valley, Ward 2, Đà Lạt, Viê...","Tọa lạc tại thành phố Đà Lạt, cách Hồ Xuân Hươ...","11.94775486,108.44124913",-3712045,thành phố đà lạt hồ xuân hương 500 m dalat win...,11.94775486,108.44124913
2,https://www.booking.com/hotel/vn/ttr-central-h...,TTR Central Apart Hotel,NaN,"9,0","4b Hải Thượng, Đà Lạt, Việt Nam","Nằm cách Quảng trường Lâm Viên 1,8 km, TTR Cen...","11.94222460,108.43152510",-3712045,nằm quảng trường lâm viên 1 8 km ttr central a...,11.94222460,108.43152510
3,https://www.booking.com/hotel/vn/raon-valley-v...,Raon Villa,NaN,"9,3","44D Trần Khánh Dư, Phường 8, Đà Lạt, Việt Nam","Tọa lạc tại thành phố Đà Lạt, cách sân golf Da...","11.96255100,108.44276300",-3712045,thành phố đà lạt sân golf dalat palace golf cl...,11.96255100,108.44276300
4,https://www.booking.com/hotel/vn/dalat-feliz-h...,Feliz Dalat Homestay,NaN,"9,5",54/13 Phan Đình Phùng (08 Nguyễn Biểu) (Dốc Nh...,Feliz Dalat Homestay cung cấp chỗ nghỉ ở vị tr...,"11.94363759,108.43447213",-3712045,feliz dalat homestay cung chỗ nghỉ vị trí lý t...,11.94363759,108.43447213
5,https://www.booking.com/hotel/vn/khach-san-hoa...,Khách sạn Hoa Sen,NaN,"8,0","103 105 phan đình phùng tp Đà Lạt, tỉnh Lâm Đồ...","Tọa lạc tại thành phố Đà Lạt, cách Quảng trườn...","11.94467500,108.43400800",-3712045,thành phố đà lạt quảng trường lâm viên 1 7 km ...,11.94467500,108.43400800
6,https://www.booking.com/hotel/vn/reddoorz-near...,Raon Dalat,NaN,"9,2","46 Tran Khanh Du, Ward 8, Đà Lạt, Việt Nam","Tọa lạc tại thành phố Đà Lạt, cách sân golf Da...","11.96217116,108.44034794",-3712045,thành phố đà lạt sân golf dalat palace golf cl...,11.96217116,108.44034794
7,https://www.booking.com/hotel/vn/raon-villa-da...,AN Boutique Hotel,NaN,"9,2","138 Bùi Thị Xuân, phường 2, thành phố Đà Lạt, ...","Tọa lạc tại thành phố Đà Lạt, cách sân golf Da...","11.94833781,108.43961489",-3712045,thành phố đà lạt sân golf dalat palace golf cl...,11.94833781,108.43961489
8,https://www.booking.com/hotel/vn/vuon-xuan-tha...,Vuon Xuan Hotel,NaN,"9,1","40 Đường Tô Hiến Thành, Đà Lạt, Việt Nam","Tọa lạc tại thành phố Đà Lạt, cách Công viên Y...","11.92828875,108.44488965",-3712045,thành phố đà lạt công viên yersin 1 6 km vuon ...,11.92828875,108.44488965
9,https://www.booking.com/hotel/vn/khach-san-tha...,ELC Dalat Hotel,NaN,"9,2","33 Đường Bà Triệu, Đà Lạt, Việt Nam","Nằm ở thành phố Đà Lạt, cách Quảng trường Lâm ...","11.93742400,108.43372300",-3712045,nằm thành phố đà lạt quảng trường lâm viên 1 5...,11.93742400,108.43372300


In [28]:
hotelx

,links,names,reviews,scores,addresses,descriptions,latitude_longitude,DEST,descriptions_clean,lat,lon
0,https://www.booking.com/hotel/vn/ruby-light-da...,Menora Grand Da Nang,NaN,"8,3","69-71-73 Do Bi Street, My Khe Beach, Ngu Hanh ...","Nằm ở thành phố Đà Nẵng, cách Bãi biển Mỹ Khê ...","16.05331759,108.24603816",-3712125,nằm thành phố đà nẵng bãi biển mỹ khê 400 m me...,16.05331759,108.24603816
1,https://www.booking.com/hotel/vn/bazan-hotel-d...,Bazan Hotel Dalat,NaN,"9,1","36 To Hien Thanh , Đà Lạt, Việt Nam","Tọa lạc tại thành phố Đà Lạt, cách Công viên Y...","11.92824835,108.44512304",-3712045,thành phố đà lạt công viên yersin 1 6 km bazan...,11.92824835,108.44512304
2,https://www.booking.com/hotel/vn/dalat-wind.vi...,Dalat Wind Hotel,NaN,"8,4","Lot R2 03-04. Golf Valley, Ward 2, Đà Lạt, Viê...","Tọa lạc tại thành phố Đà Lạt, cách Hồ Xuân Hươ...","11.94775486,108.44124913",-3712045,thành phố đà lạt hồ xuân hương 500 m dalat win...,11.94775486,108.44124913
3,https://www.booking.com/hotel/vn/ttr-central-h...,TTR Central Apart Hotel,NaN,"9,0","4b Hải Thượng, Đà Lạt, Việt Nam","Nằm cách Quảng trường Lâm Viên 1,8 km, TTR Cen...","11.94222460,108.43152510",-3712045,nằm quảng trường lâm viên 1 8 km ttr central a...,11.94222460,108.43152510
4,https://www.booking.com/hotel/vn/raon-valley-v...,Raon Villa,NaN,"9,3","44D Trần Khánh Dư, Phường 8, Đà Lạt, Việt Nam","Tọa lạc tại thành phố Đà Lạt, cách sân golf Da...","11.96255100,108.44276300",-3712045,thành phố đà lạt sân golf dalat palace golf cl...,11.96255100,108.44276300
5,https://www.booking.com/hotel/vn/dalat-feliz-h...,Feliz Dalat Homestay,NaN,"9,5",54/13 Phan Đình Phùng (08 Nguyễn Biểu) (Dốc Nh...,Feliz Dalat Homestay cung cấp chỗ nghỉ ở vị tr...,"11.94363759,108.43447213",-3712045,feliz dalat homestay cung chỗ nghỉ vị trí lý t...,11.94363759,108.43447213
6,https://www.booking.com/hotel/vn/khach-san-hoa...,Khách sạn Hoa Sen,NaN,"8,0","103 105 phan đình phùng tp Đà Lạt, tỉnh Lâm Đồ...","Tọa lạc tại thành phố Đà Lạt, cách Quảng trườn...","11.94467500,108.43400800",-3712045,thành phố đà lạt quảng trường lâm viên 1 7 km ...,11.94467500,108.43400800
7,https://www.booking.com/hotel/vn/reddoorz-near...,Raon Dalat,NaN,"9,2","46 Tran Khanh Du, Ward 8, Đà Lạt, Việt Nam","Tọa lạc tại thành phố Đà Lạt, cách sân golf Da...","11.96217116,108.44034794",-3712045,thành phố đà lạt sân golf dalat palace golf cl...,11.96217116,108.44034794
8,https://www.booking.com/hotel/vn/raon-villa-da...,AN Boutique Hotel,NaN,"9,2","138 Bùi Thị Xuân, phường 2, thành phố Đà Lạt, ...","Tọa lạc tại thành phố Đà Lạt, cách sân golf Da...","11.94833781,108.43961489",-3712045,thành phố đà lạt sân golf dalat palace golf cl...,11.94833781,108.43961489
9,https://www.booking.com/hotel/vn/vuon-xuan-tha...,Vuon Xuan Hotel,NaN,"9,1","40 Đường Tô Hiến Thành, Đà Lạt, Việt Nam","Tọa lạc tại thành phố Đà Lạt, cách Công viên Y...","11.92828875,108.44488965",-3712045,thành phố đà lạt công viên yersin 1 6 km vuon ...,11.92828875,108.44488965


In [31]:
ksf = pd.DataFrame({'links':[1],'names':[1],'reviews':[1],'scores':[1],
                      'addresses': [1],'Description':[1],'latitude_longitude': [1],'DEST': [1],'descriptions_clean':[1],'lat':[1],'lon':[1]})
    

In [32]:
ksf

,links,names,reviews,scores,addresses,Description,latitude_longitude,DEST,descriptions_clean,lat,lon
0,1,1,1,1,1,1,1,1,1,1,1


In [33]:
des = 'ádnjasfbsdkanfksdfsadf'

In [34]:
    ks = pd.DataFrame({'links':[1],'names':[1],'reviews':[1],'scores':[1],
                      'addresses': [1],'Description':[1],'latitude_longitude': [1],'DEST': [1],'descriptions_clean':[des],'lat':[1],'lon':[1]})

In [35]:
ks

,links,names,reviews,scores,addresses,Description,latitude_longitude,DEST,descriptions_clean,lat,lon
0,1,1,1,1,1,1,1,1,ádnjasfbsdkanfksdfsadf,1,1


In [44]:
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 4), min_df=0)
tfidf_matrix = tf.fit_transform(df['descriptions_clean'])
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

In [46]:
cosine_similarities

[[1.         0.10464524 0.10174804 ... 0.0606804  0.06064757 0.00361092]
 [0.10464524 1.         0.08443689 ... 0.04971149 0.02352292 0.00159125]
 [0.10174804 0.08443689 1.         ... 0.04117241 0.03439735 0.00656977]
 ...
 [0.0606804  0.04971149 0.04117241 ... 1.         0.08410422 0.00994434]
 [0.06064757 0.02352292 0.03439735 ... 0.08410422 1.         0.02301175]
 [0.00361092 0.00159125 0.00656977 ... 0.00994434 0.02301175 1.        ]]
